In [4]:
import os
import sys

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import mlflow

### tracking setup

In [39]:
# set the remote session to send the logs to
remote_server_uri = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [40]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [41]:
experiment_name = "ElasticNet_Wine"
mlflow.set_experiment(experiment_name)

2024/08/21 11:34:07 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_Wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316', creation_time=1724220247184, experiment_id='195843194857294316', last_update_time=1724220247184, lifecycle_stage='active', name='ElasticNet_Wine', tags={}>

### load the data

In [42]:
def load_data(data_path):
    data = pd.read_csv(data_path)
    train, test = train_test_split(data, test_size=0.2, random_state=42)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    train_X = train.drop(["quality"], axis=1)
    test_X = test.drop(["quality"], axis=1)
    train_y = train[["quality"]].copy()
    test_y = test[["quality"]].copy()
    return train_X, train_y, test_X, test_y

def eval_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

In [43]:
datafile_path = "../data/raw/wine-quality.csv"
X_train, y_train, X_test, y_test = load_data(datafile_path)

In [44]:
def train(alpha, l1_ratio):
    np.random.seed(42)
    with mlflow.start_run():
        net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        net.fit(X_train, y_train)

        preds = net.predict(X_test)

        rmse, mae, r2 = eval_metrics(y_test, preds)

        print("Elastic net model alpha = %.2f, l1 = %.2f" %(alpha, l1_ratio))
        print("Scores: rmse = %.4f, mae = %.4f, r2 = %.4f" %(rmse, mae, r2))

        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(datafile_path)

        print("save to {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(net, "ElasticNetModel")

In [45]:
train(alpha=0.5, l1_ratio=0.5)

Elastic net model alpha = 0.50, l1 = 0.50
Scores: rmse = 0.8220, mae = 0.6386, r2 = 0.1276
save to /mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316/c56aed0a442645429f8bdba2ab2e8931/artifacts


2024/08/21 11:34:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 11:34:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run unruly-roo-172 at: http://127.0.0.1:5000/#/experiments/195843194857294316/runs/c56aed0a442645429f8bdba2ab2e8931.
2024/08/21 11:34:11 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195843194857294316.


In [46]:
train(alpha=0.2,  l1_ratio=0.2)

Elastic net model alpha = 0.20, l1 = 0.20
Scores: rmse = 0.7874, mae = 0.6215, r2 = 0.1994
save to /mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316/900fa7d0f8f24e019f7b7e8aa31aa481/artifacts


2024/08/21 11:34:14 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 11:34:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run intrigued-robin-408 at: http://127.0.0.1:5000/#/experiments/195843194857294316/runs/900fa7d0f8f24e019f7b7e8aa31aa481.
2024/08/21 11:34:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195843194857294316.


In [47]:
train(alpha=0.1,  l1_ratio=0.1)

Elastic net model alpha = 0.10, l1 = 0.10
Scores: rmse = 0.7825, mae = 0.6182, r2 = 0.2094
save to /mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316/0e189fb01e4a4ea4b95d2e13c997f931/artifacts


2024/08/21 11:34:18 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 11:34:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-shoat-925 at: http://127.0.0.1:5000/#/experiments/195843194857294316/runs/0e189fb01e4a4ea4b95d2e13c997f931.
2024/08/21 11:34:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195843194857294316.


In [48]:
train(0.01, 0.1)

Elastic net model alpha = 0.01, l1 = 0.10
Scores: rmse = 0.7672, mae = 0.5998, r2 = 0.2400
save to /mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316/64277005834f486ab64c6826a606f8db/artifacts


2024/08/21 11:34:22 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 11:34:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run carefree-snake-582 at: http://127.0.0.1:5000/#/experiments/195843194857294316/runs/64277005834f486ab64c6826a606f8db.
2024/08/21 11:34:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195843194857294316.


#### Tagging

In [5]:
from datetime import datetime
from mlflow.tracking import MlflowClient

In [9]:
client = MlflowClient()
client.tracking_uri

'file:///mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns'

In [10]:
client.search_experiments()

[<Experiment: artifact_location='/mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316', creation_time=1724220247184, experiment_id='195843194857294316', last_update_time=1724220247184, lifecycle_stage='active', name='ElasticNet_Wine', tags={}>,
 <Experiment: artifact_location='/mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/0', creation_time=1724220229716, experiment_id='0', last_update_time=1724220229716, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
wine_experiment = client.get_experiment(experiment_id="195843194857294316")
wine_experiment

<Experiment: artifact_location='/mnt/c/Users/satyendra.mishra/Work/mle-training-mlflow/Notebook/mlruns/195843194857294316', creation_time=1724220247184, experiment_id='195843194857294316', last_update_time=1724220247184, lifecycle_stage='active', name='ElasticNet_Wine', tags={}>

In [12]:
wine_experiment_runs = client.search_runs(experiment_ids=[wine_experiment.experiment_id])

In [13]:
for run in wine_experiment_runs:
    print(run.to_dictionary()["info"]["run_name"], end="\t")
    print(run.to_dictionary()["info"]["run_id"])

intrigued-skunk-217	536fac7ab53440a5a2ba0620d8f324aa
carefree-snake-582	64277005834f486ab64c6826a606f8db
melodic-shoat-925	0e189fb01e4a4ea4b95d2e13c997f931
intrigued-robin-408	900fa7d0f8f24e019f7b7e8aa31aa481
unruly-roo-172	c56aed0a442645429f8bdba2ab2e8931


In [14]:
_run = client.get_run(run_id="536fac7ab53440a5a2ba0620d8f324aa")
_run

<Run: data=<RunData: metrics={'mae': 0.5960340126564144,
 'r2': 0.24622107331378273,
 'rmse': 0.7640559726532203}, params={'alpha': '0.0042', 'l1_ratio': '0.1'}, tags={'deployed': '21-08-24 {14-02-41.2f}',
 'mlflow.log-model.history': '[{"run_id": "536fac7ab53440a5a2ba0620d8f324aa", '
                             '"artifact_path": "ElasticNetModel", '
                             '"utc_time_created": "2024-08-21 '
                             '06:38:04.326123", "flavors": {"python_function": '
                             '{"model_path": "model.pkl", "predict_fn": '
                             '"predict", "loader_module": "mlflow.sklearn", '
                             '"python_version": "3.12.2", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.0", '
                             '"serialization_format": "cl

In [15]:
dt_var = datetime.now().strftime("%d-%m-%y {%H-%M-%S.2f}")
client.set_tag(_run.info.run_id, "deployed", dt_var)